In [1]:
import pandas as pd
import pyodbc
import os
import glob
import datetime

In [2]:
usuario='Netfeira'
senha='sqlserver'
server='192.168.0.252'
database='MOINHO'
driver='{SQL Server}'

str_conexao=(f'Driver={driver};Server={server};Database={database};UID={usuario};PWD={senha}')

In [3]:
def ConexaoSQL():
    
    try:
        
        conecta=pyodbc.connect(str_conexao)
        
        return conecta
        
        pass
    
    except Exception as erro:
        
        print('Erro: {0}'.format(erro))
        
        pass
    
    pass

In [4]:
querys={'Metas':"""

--Metas

WITH TabTpMeta(mes_ref,ano,mes,cd_prev_vda,cd_tp_prev,descricao) AS (

SELECT mes_ref,YEAR(mes_ref) AS  ano,MONTH(mes_ref) AS mes,cd_prev_vda,
tp_prev.cd_tp_prev,tp_prev.descricao
FROM prev_vda
INNER JOIN tp_prev ON prev_vda.cd_tp_prev=tp_prev.cd_tp_prev
WHERE tp_prev.descricao='VALOR'
)

SELECT ano AS 'Ano',mes AS 'ID Mês',it_prev_vda.cd_vend AS 'ID Vendedor',
valor AS 'Meta R$'
FROM it_prev_vda
INNER JOIN TabTpMeta ON TabTpMeta.cd_prev_vda=it_prev_vda.cd_prev_vda
INNER JOIN vendedor ON it_prev_vda.cd_vend=vendedor.cd_vend
AND TabTpMeta.cd_tp_prev=it_prev_vda.cd_tp_prev_det
WHERE ano=YEAR(GETDATE()) AND mes=MONTH(GETDATE())


""",'Vendas':
"""

WITH TabEvento (cd_emp,nu_ped,cd_clien,dt_criacao,cd_fila) AS (
SELECT cd_emp,nu_ped,cd_clien,CONVERT(DATETIME,CAST(dt_criacao AS date),101) AS dt_criacao,
cd_fila
FROM evento),

TabDatas (cd_emp,nu_ped,cd_clien,dt_ped,dt_fat) AS (

SELECT cd_emp,nu_ped,cd_clien,CAPV,FATU FROM (
SELECT cd_emp,nu_ped,cd_clien,dt_criacao,
fila.cd_fila
FROM TabEvento
INNER JOIN fila ON TabEvento.cd_fila=fila.cd_fila
AND fila.cd_fila IN('CAPV','FATU'))L
PIVOT(MAX(dt_criacao) FOR cd_fila IN ([CAPV],[FATU]))C
),

TabEmissao (cd_emp,nu_ped,cd_clien,dt_ped,dt_fat) AS (

SELECT cd_emp,nu_ped,cd_clien,dt_ped,
CASE WHEN dt_fat IS NULL THEN dt_ped ELSE dt_fat END AS dt_fat
FROM TabDatas),

TabItem (cd_emp,nu_ped,cd_prod,qtde,qtde_unid_vda,vl_unit_vda,vl_total,
peso_brt,peso_liq,situacao) AS (

SELECT it_pedv.cd_emp,nu_ped,it_pedv.cd_prod,qtde,qtde_unid_vda,vl_unit_vda,
ROUND(qtde_unid_vda*vl_unit_vda,2) AS vl_total,
qtde*peso_brt AS peso_brt,qtde*peso_liq AS peso_liq,
situacao
FROM it_pedv
INNER JOIN produto ON it_pedv.cd_prod=produto.cd_prod
WHERE NOT situacao IN ('CA','DV')
),

TabTipo (tp_ped,descricao,st_ped) AS (

SELECT tp_ped,descricao,
CASE WHEN bonificacao=1 THEN 'BONIFICAÇÃO' WHEN estat_com=1 
THEN 'VENDA' ELSE 'OUTROS' END AS st_ped
FROM tp_ped
WHERE bonificacao=1 OR estat_com=1
)

SELECT ped_vda.cd_emp AS 'ID Empresa',
ped_vda.nu_ped AS 'Pedido',cd_vend AS 'ID Vendedor',ped_vda.cd_clien AS 'ID Cliente',
TabEmissao.dt_ped AS 'Data do Pedido',TabEmissao.dt_fat AS 'Data do Faturamento',
TabItem.cd_prod AS 'SKU',qtde AS 'Qtde',qtde_unid_vda AS 'Qtde VDA',
vl_unit_vda AS 'Valor Unitário',vl_total AS 'Total do Pedido',
peso_brt AS 'Peso Bruto',peso_liq AS 'Peso Líquido',TabItem.situacao AS 'Situação'
FROM ped_vda
INNER JOIN TabTipo ON ped_vda.tp_ped=TabTipo.tp_ped
INNER JOIN TabEmissao ON ped_vda.nu_ped=TabEmissao.nu_ped
INNER JOIN TabItem ON ped_vda.nu_ped=TabItem.nu_ped
WHERE YEAR(TabEmissao.dt_fat)=YEAR(GETDATE())
AND MONTH(TabEmissao.dt_fat)=MONTH(GETDATE())
ORDER BY [Pedido]
        
""",'Feriado':
"""

DECLARE @DateInicial AS DATETIME,@DateFinal AS DATETIME

SET @DateInicial='2018-01-01'
SET @DateFinal=CONCAT(YEAR(GETDATE())+1,'-01-01')

;WITH TabDatas(dt_calend) AS (
SELECT @DateInicial
UNION ALL
SELECT dt_calend+1 FROM TabDatas WHERE dt_calend+1<@DateFinal),

TabCalendario (dt_calend,mes_base) AS(

SELECT CONVERT(DATETIME,CAST(dt_calend AS DATE),101),CONVERT(VARCHAR,DAY(dt_calend))+'/'+CONVERT(VARCHAR,MONTH(dt_calend)) AS mes_base
FROM TabDatas 
),

TabFeriado (dt_calend,mes_base,dias_feriado) AS (

SELECT dt_calend,mes_base,
CASE WHEN mes_base IN ('1/1','10/4','21/4','1/5','11/6','7/9','12/10','2/11','15/11','25/12','8/4') THEN 0 ELSE 1 END 
FROM TabCalendario)

SELECT DISTINCT mes_base AS 'Mês Base'
FROM TabFeriado
WHERE dias_feriado=0
ORDER BY 1
OPTION(MAXRECURSION 10000)

""",'Datas':
"""

DECLARE @DTInicial AS DATETIME,@DTFinal AS DATETIME

SET @DTInicial='2018-01-01'
SET @DTFinal= CONCAT(YEAR(GETDATE())+1,'-01-','01')

;WITH Calendario (Datas) AS(

SELECT @DTInicial
UNION ALL
SELECT Datas+1
FROM Calendario WHERE  Datas+1<@DTFinal
)

SELECT CONVERT(DATETIME,CAST(Datas AS DATE),101) AS 'Data',YEAR(Datas) AS 'Ano',MONTH(Datas) AS 'Cód. Mês',
CHOOSE(MONTH(Datas),'JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO') AS 'Mês',
CHOOSE(MONTH(Datas),'JAN','FEV','MAR','ABR','MAI','JUN','JUL','AGO','SET','OUT','NOV','DEZ') AS 'Mês Resumido',
DAY(Datas) AS 'Dia',CONVERT(VARCHAR(7),Datas,120) AS 'Mês Meta',
DATEPART(DW,Datas) AS 'Cód. Semana',CHOOSE(DATEPART(DW,Datas),'DOM','SEG','TER','QUAR','QUI','SEX','SÁB') AS 'Semana',
CASE WHEN DATEPART(DW,Datas) IN (7,1) THEN 0 ELSE 1 END AS 'Dias Úteis',
CASE WHEN MONTH(Datas)<=3 THEN '1º TRIM' WHEN MONTH(Datas)<=6 THEN '2º TRIM' WHEN MONTH(Datas)<=9 THEN '3º TRIM' WHEN MONTH(Datas)<=12 THEN '4º TRIM' END AS 'Trimestre Ano',
CASE WHEN MONTH(Datas)<=6 THEN '1º SEM' ELSE '2º SEM' END AS 'Semestre Ano',DATEPART(WEEK,Datas) AS 'Semana Ano',
CONVERT(VARCHAR,DAY(Datas))+'/'+CONVERT(VARCHAR,MONTH(Datas)) AS 'Mês Base'
FROM Calendario OPTION(MAXRECURSION 10000)

""",'Vendedores':
"""

WITH TabSupervisor (cd_equipe,descricao,nome_resumido) AS (

SELECT equipe.cd_equipe,descricao,
CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(nome)))=0 THEN nome ELSE
RTRIM(LEFT(nome,(CHARINDEX(' ',LTRIM(RTRIM(nome)))))) + ' ' + 
LTRIM(RIGHT(nome,(CHARINDEX(' ',REVERSE(LTRIM(RTRIM(nome))))))) END AS nome_resumido
FROM equipe
INNER JOIN vendedor ON equipe.cd_vend_sup=vendedor.cd_vend
WHERE equipe.ativo=1
)

SELECT cd_vend AS 'ID Vendedor',nome_gue AS 'Vendedor',
--categ.descricao AS 'Categoria',
TabSupervisor.descricao AS 'Equipe',
CASE WHEN TabSupervisor.descricao='EQUIPE 9 (ATIVO)' THEN 'PEDIDOS'
WHEN TabSupervisor.nome_resumido='JULIO DELFINO' THEN 'ROGERIO FELIPIM' ELSE
TabSupervisor.nome_resumido END AS 'Supervisor'
FROM vendedor
INNER JOIN categ ON vendedor.categ=categ.categ
INNER JOIN TabSupervisor ON vendedor.cd_equipe=TabSupervisor.cd_equipe
WHERE vendedor.ativo=1

""",'Carteira':
"""

SELECT vend_cli.cd_vend AS 'ID Vendedor',vend_cli.cd_clien AS 'ID Cliente'
FROM vend_cli
INNER JOIN cliente ON vend_cli.cd_clien=cliente.cd_clien AND cliente.ativo=1

""",'Vendas_Cliente':
"""

WITH TabEvento (cd_emp,nu_ped,cd_clien,dt_criacao,cd_fila) AS (
SELECT cd_emp,nu_ped,cd_clien,CONVERT(DATETIME,CAST(dt_criacao AS date),101) AS dt_criacao,
cd_fila
FROM evento),

TabDatas (cd_emp,nu_ped,cd_clien,dt_ped,dt_fat) AS (

SELECT cd_emp,nu_ped,cd_clien,CAPV,FATU FROM (
SELECT cd_emp,nu_ped,cd_clien,dt_criacao,
fila.cd_fila
FROM TabEvento
INNER JOIN fila ON TabEvento.cd_fila=fila.cd_fila
AND fila.cd_fila IN('CAPV','FATU'))L
PIVOT(MAX(dt_criacao) FOR cd_fila IN ([CAPV],[FATU]))C
),

TabEmissao (cd_emp,nu_ped,cd_clien,dt_ped,dt_fat) AS (

SELECT cd_emp,nu_ped,cd_clien,dt_ped,
CASE WHEN dt_fat IS NULL THEN dt_ped ELSE dt_fat END AS dt_fat
FROM TabDatas),

TabItem (cd_emp,nu_ped,cd_prod,qtde,qtde_unid_vda,vl_unit_vda,vl_total,
peso_brt,peso_liq,situacao) AS (

SELECT it_pedv.cd_emp,nu_ped,it_pedv.cd_prod,qtde,qtde_unid_vda,vl_unit_vda,
ROUND(qtde_unid_vda*vl_unit_vda,2) AS vl_total,
qtde*peso_brt AS peso_brt,qtde*peso_liq AS peso_liq,
situacao
FROM it_pedv
INNER JOIN produto ON it_pedv.cd_prod=produto.cd_prod
WHERE NOT situacao IN ('CA','DV')
),

TabTipo (tp_ped,descricao,st_ped) AS (

SELECT tp_ped,descricao,
CASE WHEN bonificacao=1 THEN 'BONIFICAÇÃO' WHEN estat_com=1 
THEN 'VENDA' ELSE 'OUTROS' END AS st_ped
FROM tp_ped
WHERE bonificacao=1 OR estat_com=1
),

TabCarteira (cd_clien,cd_vend,prioritario,cd_equipe) AS (

SELECT cd_clien,vend_cli.cd_vend,prioritario,vendedor.cd_equipe
FROM vend_cli
INNER JOIN vendedor ON vend_cli.cd_vend=vendedor.cd_vend
),

TabPedido (cd_emp,tp_ped,nu_ped,cd_clien,cd_vend,cd_equipe) AS (

SELECT ped_vda.cd_emp,tp_ped,nu_ped,ped_vda.cd_clien,ped_vda.cd_vend,
vendedor.cd_equipe
FROM ped_vda
INNER JOIN vendedor ON ped_vda.cd_vend=vendedor.cd_vend
),

TabVendCad (cd_emp,tp_ped,nu_ped,cd_clien,cd_vend,cd_equipe,vend_cad) AS (

SELECT cd_emp,tp_ped,nu_ped,cd_clien,cd_vend,cd_equipe,
CASE WHEN (
CASE WHEN (
(SELECT MAX(Tab01.cd_vend) FROM TabCarteira AS Tab01
WHERE Tab01.cd_clien=TabPedido.cd_clien AND Tab01.cd_vend=TabPedido.cd_vend
AND Tab01.cd_equipe=TabPedido.cd_equipe)) IS NOT NULL THEN
(SELECT MAX(Tab01.cd_vend) FROM TabCarteira AS Tab01
WHERE Tab01.cd_clien=TabPedido.cd_clien AND Tab01.cd_vend=TabPedido.cd_vend
AND Tab01.cd_equipe=TabPedido.cd_equipe) ELSE
(SELECT MAX(Tab01.cd_vend) FROM TabCarteira AS Tab01
WHERE Tab01.cd_clien=TabPedido.cd_clien AND Tab01.cd_equipe=TabPedido.cd_equipe)
END) IS NOT NULL THEN
CASE WHEN (
(SELECT MAX(Tab01.cd_vend) FROM TabCarteira AS Tab01
WHERE Tab01.cd_clien=TabPedido.cd_clien AND Tab01.cd_vend=TabPedido.cd_vend
AND Tab01.cd_equipe=TabPedido.cd_equipe)) IS NOT NULL THEN
(SELECT MAX(Tab01.cd_vend) FROM TabCarteira AS Tab01
WHERE Tab01.cd_clien=TabPedido.cd_clien AND Tab01.cd_vend=TabPedido.cd_vend
AND Tab01.cd_equipe=TabPedido.cd_equipe) ELSE
(SELECT MAX(Tab01.cd_vend) FROM TabCarteira AS Tab01
WHERE Tab01.cd_clien=TabPedido.cd_clien AND Tab01.cd_equipe=TabPedido.cd_equipe)
END ELSE
(SELECT MAX(Tab01.cd_vend) FROM TabCarteira AS Tab01
WHERE Tab01.cd_clien=TabPedido.cd_clien AND Tab01.prioritario=1) END
AS vend_cad
FROM TabPedido
)

SELECT TabVendCad.cd_emp AS 'ID Empresa',
TabVendCad.nu_ped AS 'Pedido',vend_cad AS 'ID Vendedor',TabVendCad.cd_clien AS 'ID Cliente',
TabEmissao.dt_ped AS 'Data do Pedido',TabEmissao.dt_fat AS 'Data do Faturamento',
TabItem.cd_prod AS 'SKU',qtde AS 'Qtde',qtde_unid_vda AS 'Qtde VDA',
vl_unit_vda AS 'Valor Unitário',vl_total AS 'Total do Pedido',
peso_brt AS 'Peso Bruto',peso_liq AS 'Peso Líquido',TabItem.situacao AS 'Situação'
FROM TabVendCad
INNER JOIN TabTipo ON TabVendCad.tp_ped=TabTipo.tp_ped
INNER JOIN TabEmissao ON TabVendCad.nu_ped=TabEmissao.nu_ped
INNER JOIN TabItem ON TabVendCad.nu_ped=TabItem.nu_ped
ORDER BY [Pedido]

""",'Cliente':
"""

WITH TabLista (cd_clien,ce,co) AS (

SELECT * FROM (
SELECT cd_clien,tp_tel
FROM tel_cli
WHERE tp_tel IN ('CE','CO')
)L
PIVOT(COUNT(tp_tel) FOR tp_tel IN ([CE],[CO]))C),

TabListaContato (cd_clien,tp_tel) AS (

SELECT cd_clien,
CASE WHEN (ce=co) OR (ce>co) then 'CE' WHEN ce<co THEN 'CO'
END AS tp_tel
FROM TabLista),

TabContato (cd_clien,ddd,numero) AS (

SELECT TabListaContato.cd_clien,ddd,numero
FROM TabListaContato
INNER JOIN tel_cli ON TabListaContato.cd_clien=tel_cli.cd_clien
AND TabListaContato.tp_tel=tel_cli.tp_tel and tel_cli.seq=1)

SELECT cliente.cd_clien AS 'ID Cliente',nome_res AS 'Nome Fantasia',cgc_cpf AS 'CNPJ',
CASE WHEN LEN(cgc_cpf)=14 THEN 'CNPJ' ELSE 'CPF' END AS 'Tipo de Cliente',
ddd AS 'DDD',CONVERT(VARCHAR,numero) AS 'Número'
FROM cliente
LEFT JOIN TabContato ON cliente.cd_clien=TabContato.cd_clien
WHERE ativo=1

"""
}

In [5]:
conectando=ConexaoSQL()

In [25]:
vendas_df=pd.read_sql(querys['Vendas'],conectando)

vd_cliente=pd.read_sql(querys['Vendas_Cliente'],conectando)

vendedor_df=pd.read_sql(querys['Vendedores'],conectando)

cliente_df=pd.read_sql(querys['Cliente'],conectando)

In [7]:
vendedor_df.columns

Index(['ID Vendedor', 'Vendedor', 'Equipe', 'Supervisor'], dtype='object')

In [8]:
vendas_df.columns

Index(['ID Empresa', 'Pedido', 'ID Vendedor', 'ID Cliente', 'Data do Pedido',
       'Data do Faturamento', 'SKU', 'Qtde', 'Qtde VDA', 'Valor Unitário',
       'Total do Pedido', 'Peso Bruto', 'Peso Líquido', 'Situação'],
      dtype='object')

In [9]:
vendedor_df['Pedido']=vendedor_df.apply(lambda info: len(vendas_df['Pedido'].loc[vendas_df['ID Vendedor']==info['ID Vendedor']].unique().tolist()),axis=1)

In [10]:
vendas_df['Ano']=vendas_df['Data do Faturamento'].dt.year

vendas_df['ID Mês']=vendas_df['Data do Faturamento'].dt.month

vendas_df['Dia']=vendas_df['Data do Faturamento'].dt.day

data_max=vendas_df['Data do Faturamento'].loc[(vendas_df['Ano']==2022)&~(vendas_df['ID Mês']==4)].max()

data_min=data_max-datetime.timedelta(days=120)

In [11]:
temp_df=pd.DataFrame()

vda_df=pd.DataFrame()

temp_df=vendas_df[['ID Cliente','ID Vendedor','Total do Pedido']].loc[vendas_df['Data do Faturamento'].between(data_min,data_max)].groupby(['ID Cliente','ID Vendedor']).sum(['Total do Pedido']).reset_index()

vda_df=vendas_df[['ID Cliente','ID Vendedor','Total do Pedido']].loc[(vendas_df['Ano']==2022)&(vendas_df['ID Mês']==4)].groupby(['ID Cliente','ID Vendedor']).sum().reset_index()

temp_df['Realizado R$']=temp_df.apply(lambda info: vda_df['Total do Pedido'].loc[(vda_df['ID Cliente']==info['ID Cliente'])&(vda_df['ID Vendedor']==info['ID Vendedor'])].sum(),axis=1)

temp_df['Status']=temp_df.apply(lambda info: 'SEM COMPRA' if info['Realizado R$']==0 else 'COMPROU',axis=1)

In [12]:
temp_df.loc[temp_df['Status']=='SEM COMPRA']

,ID Cliente,ID Vendedor,Total do Pedido,Realizado R$,Status
0,10,AVULSO,441388.80,0.0,SEM COMPRA
24,2693,MARCOSLI,3245.02,0.0,SEM COMPRA
28,2745,RAQUELSO,207.36,0.0,SEM COMPRA
29,2943,LOJA,1673.60,0.0,SEM COMPRA
30,2944,LOJA,100.11,0.0,SEM COMPRA
...,...,...,...,...,...
3951,27546,ADELIALO,209.04,0.0,SEM COMPRA
3952,27547,ADELIALO,434.70,0.0,SEM COMPRA
3953,27548,CELSOCAR,114.04,0.0,SEM COMPRA
3954,27549,AVULSO,503.40,0.0,SEM COMPRA


In [14]:
dados_df=cliente_df.merge(temp_df,on='ID Cliente',how='inner')

In [20]:
df=dados_df.merge(vendedor_df,on='ID Vendedor',how='inner')

In [23]:
df[['ID Cliente', 'Nome Fantasia', 'CNPJ', 'Tipo de Cliente', 'DDD','Número','ID Vendedor','Vendedor', 'Equipe', 'Supervisor','Total do Pedido', 'Realizado R$', 'Status']]

,ID Cliente,Nome Fantasia,CNPJ,Tipo de Cliente,DDD,Número,ID Vendedor,Vendedor,Equipe,Supervisor,Total do Pedido,Realizado R$,Status
0,10,DE MARCHI,52884061000162,CNPJ,11,48158400,AVULSO,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM,441388.80,0.00,SEM COMPRA
1,989,MCCAIN DO BRASIL,68090240000168,CNPJ,11,38483632,AVULSO,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM,2633.78,238.27,COMPROU
2,10686,VAREJINHO,05209965000102,CNPJ,11,38314933,AVULSO,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM,2738.84,375.06,COMPROU
3,19939,CONSUMIDOR FINAL,22222222222,CPF,11,11111111,AVULSO,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM,163921.17,29215.01,COMPROU
4,20734,HAVANNA JUNDIAI,28673201000179,CNPJ,11,45955919,AVULSO,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM,796.68,0.00,SEM COMPRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,24240,SUPER JOANIN LJ 13,59276790002048,CNPJ,11,22117030,ADIVAL,ADIVAL,EQUIPE 2 (REP),ULISSES BACCI,11316.12,0.00,SEM COMPRA
3956,24446,SUPER JOANIN LJ 21,59276790002552,CNPJ,11,43353400,ADIVAL,ADIVAL,EQUIPE 2 (REP),ULISSES BACCI,7571.56,0.00,SEM COMPRA
3957,24854,SUPER JOANIN LJ 04,59276790002471,CNPJ,11,29172755,ADIVAL,ADIVAL,EQUIPE 2 (REP),ULISSES BACCI,4261.24,0.00,SEM COMPRA
3958,24882,SUPER JOANIN LJ 22,59276790002633,CNPJ,11,43353400,ADIVAL,ADIVAL,EQUIPE 2 (REP),ULISSES BACCI,2110.34,0.00,SEM COMPRA


In [30]:
vendedor_df['Equipe'].loc[vendedor_df['Supervisor'].str.contains('ULISSES')].unique().tolist()

['EQUIPE 2 (REP)', 'EQUIPE 3 (CLT)', 'EQUIPE 10 (REP)']

In [60]:
vendas_df.merge(vendedor_df,on='ID Vendedor',how='inner')

,ID Empresa,Pedido,ID Vendedor,ID Cliente,Data do Pedido,Data do Faturamento,SKU,Qtde,Qtde VDA,Valor Unitário,Total do Pedido,Peso Bruto,Peso Líquido,Situação,Vendedor,Equipe,Supervisor
0,1,388423,AVULSO,19939,2022-04-01,2022-04-01,47,2.0,2.0,21.76,43.52,2.52,2.40,FA,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM
1,1,388423,AVULSO,19939,2022-04-01,2022-04-01,478,2.0,2.0,23.13,46.26,2.52,2.40,FA,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM
2,1,388424,AVULSO,19939,2022-04-01,2022-04-01,1078,5.0,5.0,13.81,69.05,5.25,5.00,FA,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM
3,1,388424,AVULSO,19939,2022-04-01,2022-04-01,1080,1.0,1.0,53.31,53.31,1.26,1.20,FA,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM
4,1,388424,AVULSO,19939,2022-04-01,2022-04-01,618,1.0,1.0,37.21,37.21,1.02,1.02,FA,AVULSO,EQUIPE 8 (ATEND),ROGERIO FELIPIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15370,1,392971,CARLOSH,11416,2022-04-20,2022-04-22,403,1.0,1.0,20.42,20.42,1.05,1.00,FA,CARLOSH,EQUIPE 6 (CLT),CARLOS PASSOS
15371,1,392971,CARLOSH,11416,2022-04-20,2022-04-22,405,1.0,1.0,47.14,47.14,1.05,1.00,FA,CARLOSH,EQUIPE 6 (CLT),CARLOS PASSOS
15372,1,392971,CARLOSH,11416,2022-04-20,2022-04-22,666,1.0,1.0,17.99,17.99,1.05,1.00,FA,CARLOSH,EQUIPE 6 (CLT),CARLOS PASSOS
15373,1,392971,CARLOSH,11416,2022-04-20,2022-04-22,1045,1.0,1.0,19.70,19.70,1.05,1.00,FA,CARLOSH,EQUIPE 6 (CLT),CARLOS PASSOS


In [58]:
dados=glob.glob('*.xlsx')

email_df=pd.read_excel(dados[0])

supervisor=email_df['Supervisor'].tolist()

for sup in supervisor:
    
    cargo=email_df['Status'].loc[email_df['Supervisor']==sup].tolist()
    
    if(str(cargo[0]).upper()=='SUPERVISOR'):
        
        print(sup)
        
        pass
    
    pass

ULISSES BACCI
DOUGLAS TEDESCHI
CARLOS PASSOS
PEDIDOS
